# Exploratory Data Analysis

Validating GSS data

Allen Downey

[MIT License](https://en.wikipedia.org/wiki/MIT_License)

In [83]:
import pandas as pd
import numpy as np

from distribution import Pmf

## Loading and validation


Reading the Excel file generates an error that seems to be a known issue.

https://github.com/python-excel/xlrd/issues/218

It doesn't look like anyone is working on it from the Python end, so I wonder if there's a way to work around it by generating a Python-friendly xls file.
    

In [84]:
pd.read_excel('gss_eda/GSS.xls')

ERROR *** codepage 21010 -> encoding 'unknown_codepage_21010' -> LookupError: unknown encoding: unknown_codepage_21010


LookupError: unknown encoding: unknown_codepage_21010

Python just doesn't recognise the SAS file, in either of the formats it expects.

In [85]:
pd.read_sas('gss_eda/GSS.sas', format='xport')

ValueError: Header record is not an XPORT file.

In [86]:
pd.read_sas('gss_eda/GSS.sas', format='sas7bdat')

ValueError: magic number mismatch (not a SAS file?)

Trying to read the Stata file, it looks like Pandas doesn't recognize the format.

In [87]:
pd.read_stata('gss_eda/GSS.dat')

ValueError: Version of given Stata file is not 104, 105, 108, 111 (Stata 7SE), 113 (Stata 8/9), 114 (Stata 10/11), 115 (Stata 12), 117 (Stata 13), or 118 (Stata 14)

In [46]:
import re

class FixedWidthVariables(object):
    """Represents a set of variables in a fixed width file."""

    def __init__(self, variables, index_base=0):
        """Initializes.

        variables: DataFrame
        index_base: are the indices 0 or 1 based?

        Attributes:
        colspecs: list of (start, end) index tuples
        names: list of string variable names
        """
        self.variables = variables

        # note: by default, subtract 1 from colspecs
        self.colspecs = variables[['start', 'end']] - index_base

        # convert colspecs to a list of pair of int
        self.colspecs = self.colspecs.astype(np.int).values.tolist()
        self.names = variables['name']

    def ReadFixedWidth(self, filename, **options):
        """Reads a fixed width ASCII file.

        filename: string filename

        returns: DataFrame
        """
        df = pd.read_fwf(filename,
                         colspecs=self.colspecs, 
                         names=self.names,
                         **options)
        return df


def ReadStataDct(dct_file, **options):
    """Reads a Stata dictionary file.

    dct_file: string filename
    options: dict of options passed to open()

    returns: FixedWidthVariables object
    """
    type_map = dict(byte=int, int=int, long=int, float=float, 
                    double=float, numeric=float)

    var_info = []
    with open(dct_file, **options) as f:
        for line in f:
            match = re.search( r'_column\(([^)]*)\)', line)
            if not match:
                continue
            start = int(match.group(1))
            t = line.split()
            vtype, name, fstring = t[1:4]
            name = name.lower()
            if vtype.startswith('str'):
                vtype = str
            else:
                vtype = type_map[vtype]
            long_desc = ' '.join(t[4:]).strip('"')
            var_info.append((start, vtype, name, fstring, long_desc))
            
    columns = ['start', 'type', 'name', 'fstring', 'desc']
    variables = pd.DataFrame(var_info, columns=columns)

    # fill in the end column by shifting the start column
    variables['end'] = variables.start.shift(-1)
    variables.loc[len(variables)-1, 'end'] = 0

    dct = FixedWidthVariables(variables, index_base=1)
    return dct

def read_gss(dirname):
    """Reads GSS files from the given directory.
    
    dirname: string
    
    returns: DataFrame
    """
    dct = ReadStataDct(dirname + '/GSS.dct')
    gss = dct.ReadFixedWidth(dirname + '/GSS.dat.gz',
                             compression='gzip')
    return gss

## Read extract

https://gssdataexplorer.norc.org/projects/52787/extracts

In [47]:
gss = read_gss('gss_eda')
print(gss.shape)
gss.head()

(62466, 101)


,year,id_,agewed,divorce,sibs,childs,age,educ,paeduc,maeduc,...,memchurh,realinc,cohort,marcohrt,ballot,wtssall,adults,compuse,databank,wtssnr
0,1972,1,0,0,3,0,23,16,10,97,...,0,18951.0,1949,0,0,0.4446,1,0,0,1.0
1,1972,2,21,2,4,5,70,10,8,8,...,0,24366.0,1902,1923,0,0.8893,2,0,0,1.0
2,1972,3,20,2,5,4,48,12,8,8,...,0,24366.0,1924,1944,0,0.8893,2,0,0,1.0
3,1972,4,24,2,5,0,27,17,16,12,...,0,30458.0,1945,1969,0,0.8893,2,0,0,1.0
4,1972,5,22,2,2,2,61,12,8,8,...,0,50763.0,1911,1933,0,0.8893,2,0,0,1.0


In [48]:
gss[gss.year==2016].adults.value_counts().sort_index()

1     994
2    1436
3     317
4      88
5      23
6       6
8       2
9       1
Name: adults, dtype: int64

In [49]:
def replace_invalid(df):
    df.realinc.replace([0], np.nan, inplace=True)                  
    df.educ.replace([98,99], np.nan, inplace=True)
    # 89 means 89 or older
    df.age.replace([98, 99], np.nan, inplace=True) 
    df.cohort.replace([9999], np.nan, inplace=True)
    df.adults.replace([9], np.nan, inplace=True)

replace_invalid(gss)

The proportion of women in this dataset is higher than in the population, even after weighting.

In [50]:
sex = gss.loc[gss.year==2010, 'sex']
sex.value_counts()

2    1153
1     891
Name: sex, dtype: int64

In [51]:
pmf = Pmf([1,2])
pmf[1] = np.sum(sex==1)
pmf[2] = np.sum(sex==2)
pmf.normalize()
pmf

1    0.43591
2    0.56409
Name: Pmf, dtype: float64

In [52]:
pmf = Pmf([1,2])
pmf[1] = np.sum((sex==1) * gss.wtssall)
pmf[2] = np.sum((sex==2) * gss.wtssall)
pmf.normalize()
pmf

1    0.451634
2    0.548366
Name: Pmf, dtype: float64

In [53]:
pmf = Pmf([1,2])
pmf[1] = np.sum((sex==1) * gss.wtssnr)
pmf[2] = np.sum((sex==2) * gss.wtssnr)
pmf.normalize()
pmf

1    0.453784
2    0.546216
Name: Pmf, dtype: float64

In [54]:
pmf = Pmf([1,2])
pmf[1] = np.sum((sex==1) * gss.wtssall * gss.adults)
pmf[2] = np.sum((sex==2) * gss.wtssall * gss.adults)
pmf.normalize()
pmf

1    0.463868
2    0.536132
Name: Pmf, dtype: float64

In [55]:
pmf = Pmf([1,2])
pmf[1] = np.sum((sex==1) * gss.wtssnr * gss.adults)
pmf[2] = np.sum((sex==2) * gss.wtssnr * gss.adults)
pmf.normalize()
pmf

1    0.464468
2    0.535532
Name: Pmf, dtype: float64

In [72]:
pmf = Pmf([1,2])
pmf[1] = np.sum((sex==1) * gss.wtssall * 1.145)
pmf[2] = np.sum((sex==2) * gss.wtssall)
pmf.normalize()
pmf

1    0.485338
2    0.514662
Name: Pmf, dtype: float64

According to Census data, here are the number of male and female adults in the U.S. in 2010 (estimated population)

https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?src=bkmk

In [64]:
pmf = Pmf([1,2])
pmf[1] = 114173831
pmf[2] = 121043794
pmf.normalize()
pmf

1    0.485397
2    0.514603
Name: Pmf, dtype: float64

In [40]:
gss.loc[gss.year==2010, 'wtssall'].describe()

count    2044.000000
mean        1.000000
std         0.543717
min         0.463128
25%         0.463128
50%         0.926255
75%         0.926255
max         4.210252
Name: wtssall, dtype: float64

In [41]:
gss.loc[gss.year==2010, 'wtssnr'].describe()

count    2044.000000
mean        1.000000
std         0.575564
min         0.336970
25%         0.552393
50%         0.874341
75%         1.192883
max         4.733329
Name: wtssnr, dtype: float64

In [42]:
gss.loc[gss.year==2010, 'adults'].describe()

count    2038.000000
mean        1.851325
std         0.852370
min         1.000000
25%         1.000000
50%         2.000000
75%         2.000000
max         7.000000
Name: adults, dtype: float64

In [44]:
gss.loc[gss.year==2010, 'adults'].value_counts().sort_index()

1.0     728
2.0    1022
3.0     189
4.0      70
5.0      22
6.0       5
7.0       2
Name: adults, dtype: int64